In [14]:
from scipy.sparse import rand
import numpy as np
import tensorflow as tf
from parallelrank import find_rank

assert len(tf.config.list_physical_devices('GPU')) > 0

matrix = rand(1000, 1000, density=0.2, format='csr', dtype=np.int8)
matrix.data[:] = 1

In [19]:
%%time
rank = tf.linalg.matrix_rank(matrix.todense(), tol=1e-10).numpy()
print(rank)

1000
CPU times: user 463 ms, sys: 49.5 ms, total: 512 ms
Wall time: 513 ms


In [20]:
%%time
rank = find_rank(matrix.indptr, matrix.indices, matrix.shape[1], matrix.shape[0])
print(rank)

1000
CPU times: user 2.61 s, sys: 123 ms, total: 2.73 s
Wall time: 2.74 s
